In [1]:
import numpy as np
import pandas as pd
from toolz.functoolz import memoize, partial
from tqdm import tqdm, trange
tqdm = partial(tqdm, leave=True, position=0)

%load_ext line_profiler

max_level = 60
max_stat = 89

def stat_cost(stat):
    if stat <= 1:
        return 0
    else:
        return 2 + (stat-3)//10

@memoize
def total_stat_cost(stat):
    if stat <= 1:
        return 0
    else:
        return total_stat_cost(stat-1) + stat_cost(stat)
    
@memoize
def stat_limit(points):
    stat = 1
    while total_stat_cost(stat+1) <= points:
        stat += 1
    return stat

def point_gain(level):
    if level <= 1:
        return 0
    else:
        return 3 + level//5
    
@memoize
def level_points(level):
    if level <= 1:
        return 0
    else:
        return level_points(level-1) + point_gain(level)

def required_points(stats):
    return sum(map(total_stat_cost, stats.values()))

@memoize
def required_level_for_points(points):
    level = 1
    current_points = 0
    while current_points < points:
        level += 1
        current_points += point_gain(level)
    return level

def required_level(stats):
    req_points = required_points(stats)
    return required_level_for_points(req_points)


In [2]:
def player_stats(
    level,
    stats,
    weapon_atk=73,
    atk_mult=1+0.05,
    armor_crit_mult=0.05+0.05,
    armor_crit_rate=0,
    armor_dodge_rate=0,
    armor_healing=0,
    jewelery_atk=0,
#     jewelery_atk=16,
    jewelery_crit_mult=0,
    jewelery_crit_rate=0.1,
#     jewelery_crit_rate=0,
    jewelery_dodge_rate=0,
    jewelery_healing=0,
#     jewelery_healing=-0.05,
    lifesteal=0.05+0.01+0.03,
):
    player = {}
    
    player['Attack'] = (4 + level + stats['STR'] + weapon_atk + jewelery_atk) * (atk_mult)

    player['Crit Mult'] = 1.5 + 0.05*(stats['STR']//10) + armor_crit_mult + jewelery_crit_mult
    player['Crit Rate'] = 0.1 + 0.05*(stats['AGI']//10) + armor_crit_rate + jewelery_crit_rate
    player['Dodge Rate'] = 0.1 + 0.005*stats['AGI'] + armor_dodge_rate + jewelery_dodge_rate
    player['Healing Bonus'] = 1 + stats['INT']/50 + armor_healing + jewelery_healing
    player['Magic Resist'] = 0.05*(stats['INT']//10)
    player['Physical Resist'] = 0.05*(stats['VIT']//10)
    
    player['Avg Damage'] = player['Attack'] * (player['Crit Mult']*player['Crit Rate'] + (1-player['Crit Rate']))
    player['Avg Lifesteal'] = player['Healing Bonus'] * lifesteal * player['Avg Damage']
#     player['Avg Lifesteal'] *= 1 + player['Dodge Rate']
#     player['Avg Lifesteal'] *= ((1+player['Magic Resist'])+(1+player['Physical Resist']))/2
    
    return player


level = 48
stats = {
    'STR': 45,
    'AGI': 10,
    'INT': 40,
    'VIT': 20,
}
display(player_stats(level, stats))

{'Attack': 178.5,
 'Crit Mult': 1.8,
 'Crit Rate': 0.25,
 'Dodge Rate': 0.15000000000000002,
 'Healing Bonus': 1.8,
 'Magic Resist': 0.2,
 'Physical Resist': 0.1,
 'Avg Damage': 214.2,
 'Avg Lifesteal': 34.7004}

In [8]:
def optimal_builds(fixed=None, max_level=60, max_stat=89):
    attributes = ['STR', 'AGI', 'INT', 'VIT']
    stats = {attr: 1 for attr in attributes}
    max_points = level_points(max_level)
    if fixed:
        for attr in fixed:
            attributes.remove(attr)
            stats[attr] = fixed[attr]
    
    builds = []
    
    def allocate(idx=0):
        attr = attributes[idx]
        available_points = max_points - required_points(stats)
        
        stat_levels = range(1, stat_limit(available_points)+1)
        if idx == 0:
            stat_levels = tqdm(stat_levels)
        for i in stat_levels:
            stats[attr] = i
            
            if idx == len(attributes)-1:
                build = stats.copy()
                build['Point Cost'] = required_points(stats)
                build['Level'] = required_level_for_points(build['Point Cost'])
                build.update(player_stats(build['Level'], stats))
                builds.append(build)
            else:
                allocate(idx=idx+1)
        stats[attr] = 1
        
    allocate()
    
    builds = pd.DataFrame(builds)
    builds = builds.set_index('Level')
#     builds = builds.sort_values(['Metric', 'Point Cost'], ascending=[True, False])
#     builds = builds.groupby('Level').last()
    
    return builds
    
                                      
fixed = {
#     'INT': 40,
    'VIT': 20,
}
# fixed = None

builds = optimal_builds(fixed=fixed)
# display(builds.sort_values(['Avg Damage', 'Point Cost'], ascending=[True, False]).groupby('Level').last())
display(builds.sort_values(['Avg Lifesteal', 'Point Cost'], ascending=[True, False]).groupby('Level').last())

100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 93.04it/s]


,STR,AGI,INT,VIT,Point Cost,Attack,Crit Mult,Crit Rate,Dodge Rate,Healing Bonus,Magic Resist,Physical Resist,Avg Damage,Avg Lifesteal
Level,,,,,,,,,,,,,,
13,2,1,3,20,49,96.60,1.60,0.20,0.105,1.06,0.00,0.1,108.192000,10.321517
14,1,1,6,20,54,96.60,1.60,0.20,0.105,1.12,0.00,0.1,108.192000,10.905754
15,1,1,9,20,60,97.65,1.60,0.20,0.105,1.18,0.00,0.1,109.368000,11.614882
16,1,1,12,20,66,98.70,1.60,0.20,0.105,1.24,0.05,0.1,110.544000,12.336710
17,3,1,13,20,72,101.85,1.60,0.20,0.105,1.26,0.05,0.1,114.072000,12.935765
18,3,1,15,20,78,102.90,1.60,0.20,0.105,1.30,0.05,0.1,115.248000,13.484016
19,3,1,17,20,84,103.95,1.60,0.20,0.105,1.34,0.05,0.1,116.424000,14.040734
20,11,1,14,20,91,113.40,1.65,0.20,0.105,1.28,0.05,0.1,128.142000,14.761958
21,10,1,17,20,98,113.40,1.65,0.20,0.105,1.34,0.05,0.1,128.142000,15.453925


In [47]:
display(builds.sort_values(['Avg Damage', 'Point Cost'], ascending=[True, False]).groupby('Level').last())
display(builds.sort_values(['Avg Lifesteal', 'Point Cost'], ascending=[True, False]).groupby('Level').last())

,STR,AGI,INT,VIT,Point Cost,Attack,Crit Mult,Crit Rate,Dodge Rate,Healing Bonus,Avg Damage,Avg Lifesteal
Level,,,,,,,,,,,,
13,3,1,1,20,48,114.45,1.60,0.10,0.105,-0.03,121.317000,-0.327556
14,6,1,1,20,54,118.65,1.60,0.10,0.105,-0.03,125.769000,-0.339576
15,9,1,1,20,60,122.85,1.60,0.10,0.105,-0.03,130.221000,-0.351597
16,12,1,1,20,66,127.05,1.65,0.10,0.105,-0.03,135.308250,-0.365332
17,14,1,1,20,72,130.20,1.65,0.10,0.105,-0.03,138.663000,-0.374390
18,16,1,1,20,78,133.35,1.65,0.10,0.105,-0.03,142.017750,-0.383448
19,18,1,1,20,84,136.50,1.65,0.10,0.105,-0.03,145.372500,-0.392506
20,20,1,1,20,90,139.65,1.70,0.10,0.105,-0.03,149.425500,-0.403449
21,22,1,1,20,96,142.80,1.70,0.10,0.105,-0.03,152.796000,-0.412549


,STR,AGI,INT,VIT,Point Cost,Attack,Crit Mult,Crit Rate,Dodge Rate,Healing Bonus,Avg Damage,Avg Lifesteal
Level,,,,,,,,,,,,
13,2,1,3,20,49,113.40,1.60,0.10,0.105,0.01,120.20400,0.108184
14,1,1,6,20,54,113.40,1.60,0.10,0.105,0.07,120.20400,0.757285
15,1,1,9,20,60,114.45,1.60,0.10,0.105,0.13,121.31700,1.419409
16,1,1,12,20,66,115.50,1.60,0.10,0.105,0.19,122.43000,2.093553
17,1,1,14,20,72,116.55,1.60,0.10,0.105,0.23,123.54300,2.557340
18,1,1,16,20,78,117.60,1.60,0.10,0.105,0.27,124.65600,3.029141
19,1,1,18,20,84,118.65,1.60,0.10,0.105,0.31,125.76900,3.508955
20,2,1,20,20,91,120.75,1.60,0.10,0.105,0.35,127.99500,4.031842
21,2,1,22,20,97,121.80,1.60,0.10,0.105,0.39,129.10800,4.531691


In [ ]:
def optimize(fitness=avg_damage, fixed=None):
    optimize = ['STR', 'AGI', 'INT', 'VIT']
    stats = {stat: 1 for stat in optimize}
    if fixed:
        for stat in fixed:
            optimize.remove(stat)
            stats[stat] = fixed[stat]
            
    def next_point_efficiency(stats):
        efficiency = []
        current_fitness = fitness(level, stats)
        for stat in optimize:
            new_stats = stats.copy()
            new_stats[stat] += 1
            fitness_increase = fitness(level, new_stats) - current_fitness
            efficiency.append(fitness_increase / stat_cost(new_stats[stat]))
        return efficiency
    
    def best_next_stat(stats):
        return optimize[np.argmax(next_point_efficiency(stats))]
            
    optimal = pd.DataFrame(columns=stats.keys())
    optimal.index.name = 'LV'
    
    level = required_level(stats)  # starting level
    unallocated_points = level_points(level) - required_points(stats)
    while level <= 60:
        next_stat = best_next_stat(stats)
        while stat_cost(stats[next_stat]+1) <= unallocated_points:
            stats[next_stat] += 1
            unallocated_points -= stat_cost(stats[next_stat]+1)
            next_stat = best_next_stat(stats)
        optimal = optimal.append(pd.Series(stats, name=level))
        level += 1
        unallocated_points += point_gain(level)
            
    return optimal